# Transformer Architesture


In [ ]:
# Tokenizer
## Load the tokenizer from BERT as a start point
## Tokenization takes the text and maps input sequences to numbers.
## Tokienizer maps words to number IDs
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text = ['The airplane flies across the sky.','Time flies by very fast.', 'Fruit flies are nasty.']
inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt', add_special_tokens=True)
inputs




In [ ]:
## Embedding
## Embedding turns mapping of the input text vector to the embedding matrix.
## represnented rltship btwn tokens
from torch import nn
from transformers import AutoConfig
config = AutoConfig.from_pretrained('bert-base-uncased')
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)



In [ ]:
token_emb

In [ ]:
input_embeds = token_emb(inputs.input_ids)

# print(input_embeds[0][3], input_embeds[1][2], input_embeds[2][2])
#

## Notice that embedding for [CLS] and [SEP] are the same, embedding vectors for the same words are the same, regardless of meaning/context.
## There is embedding for padding and punctuations too
##

In [ ]:
print(input_embeds[0][3].shape) ## 768 dimensions in the embeddings

In [ ]:
input_embeds.shape

## [batch size: 3 bc 3 sentences,
## seq_len: 9 because longest sentence length is 6 + 1 for punctuation + 2 for begining and ending token CLS and SEP]
## hidden_dim: Bert hidden dimention for each embedding for each word

In [ ]:
## Attention head
import torch
from math import sqrt
import torch.nn.functional as F

def scaled_doct_product_attention(query, key,value):

  dim_k = key.size(-1) ## embedding size 768
  scores = torch.bmm(query, key.transpose(1,2))/sqrt(dim_k)
  weights = F.softmax(scores, dim=1)
  attn_outputs = torch.bmm(weights,value)
  return attn_outputs


In [ ]:
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self,hidden_state):
    attn_outputs = scaled_doct_product_attention(self.q(hidden_state), self.k(hidden_state),self.v(hidden_state))
    return attn_outputs



In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    num_heads = config.num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)


  def forward(self,hidden_state):
    # print(hidden_state.shape)
    # for h in self.heads:
    #   print(h(hidden_state)[0][0].shape)
    x = torch.cat([h(hidden_state) for h in self.heads],dim=-1)
    # print(x)
    # print(x.shape)
    x = self.output_linear(x)
    # print(x.shape)
    return x





In [ ]:
multihead_attn = MultiHeadAttention(config)

In [ ]:
config


In [ ]:
multihead_attn

In [ ]:
attn_output = multihead_attn(input_embeds)

In [ ]:
class FeedForward(nn.Module):
  def __init__(self,config):
    super().__init__()
    self.linear1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
  def forward(self,x):
    x = self.linear1(x)
    x = self.gelu(x)
    x = self.linear2(x)
    x = self.dropout(x)
    return x


In [ ]:
feed_forward = FeedForward(config)

In [ ]:
ff_outputs = feed_forward(attn_output)

In [ ]:
ff_outputs.shape

In [ ]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.layer_norm1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm1(x)
    x = x+self.attention(hidden_state)
    x+=self.feed_forward(self.layer_norm2(x))
    return x



In [ ]:
encoder_layer = TransformerEncoderLayer(config)
input_embeds.shape, encoder_layer(input_embeds).shape

In [ ]:
class Embeddings(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
    self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
    self.dropout = nn.Dropout()

  def forward(self, input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype = torch.long).unsqueeze(0)
    token_embeddings = self.token_embeddings(input_ids)
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = token_embeddings+position_embeddings
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings


In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

In [ ]:
inputs.input_ids

In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList(
        [TransformerEncoderLayer(config) for _ in range(config.num_hidden_layers)]
    )

  def forward(self,x):
    x = self.embeddings(x)
    for layer in self.layers:
      x = layer(x)
    return x

In [ ]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

In [ ]:
## Adding a classification head
class TransformerForSequenceClassification(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.encoder = TransformerEncoder(config)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)

  def forward(self,x):
    x = self.encoder(x)[:,0,:] ## select hidden state of [CLS] token
    x = self.dropout(x)
    x = self.classifier(x)
    return x

In [ ]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

In [ ]:
encoder_classifier(inputs.input_ids)

In [ ]:
## decoder
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask

In [ ]:
scores

In [ ]:
scores.masked_fill(mask==0,-float('inf'))

In [ ]:
def scaled_doct_product_attention(query, key, value, mask = None):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0,-float('inf'))
  weights = F.softmax(scores, dim = -1)
  return weights.bmm(value)

In [ ]:
## GPT archtesture and trainer